In [ ]:
using Random
using Flux
using PyPlot
using Distributions
using Gen
using LinearAlgebra

include("mixture_of_normals.jl")

In [ ]:
#Hyperparameters
σ = 0.002
N = 720
sig = 3;

In [ ]:
#Real data for discriminator
function real_data(N::Int, modes::Int, σ::Float64)
    μ₁ = [0.25, 0.25]
    μ₂ = [0.25, 0.75]
    μ₃ = [0.75, 0.75]
    μ₄ = [0.75, 0.25]
    μ₅ = [1.25, 1.25]
    μ₆ = [1.25, 1.75]
    μ₇ = [1.75, 1.75]
    μ₈ = [1.75, 1.25]
    μ = [μ₁, μ₂, μ₃, μ₄, μ₅, μ₆, μ₇, μ₈]
    Σ = [[σ, 0] [0, σ]]
    
    all_samples = zeros(Float64, (N*modes, 2))
    classes = zeros(Int, (N*modes))
    
    for i = 1:modes
        dist = MvNormal(μ[i], Σ)
        sample = rand(dist, N)::Matrix
        scatter(sample[1,:],sample[2,:])
        all_samples[(i-1)*N+1:i*N,:] = transpose(sample)
        classes[(i-1)*N+1:i*N] = fill(i, N)
    end
    return all_samples, classes
end
    
data, classes = real_data(2000, 2, σ);

In [ ]:
#Latent Space Sampler - 2D (Avoid using for now)
function latent_sampler(N::Int, modes::Int, σ::Float64 = 0.1)
    batch_size::Int = Int(N/modes)
    all_samples = zeros(Float64, (N*modes,2))
    μ = [float(-2.5), float(2.5)]
    Σ = [[σ, 0] [0, σ]]
    
    #Fill total samples
    total_samples = batch_size * modes
    remainder = N - total_samples
    
    for i in 1:modes
        factor = floor((3+i)/4)
        #display(factor)
        μ₁ = factor * μ[Int((floor((i+2)/2))%2 + 1)]
        μ₂ = factor * μ[Int((floor(((i+2)%2)+1)))]
        #display(μ₁)
        #display(μ₂)
        loc = [μ₁, μ₂]
        
        dist = MvNormal(loc, Σ)
        sample = rand(dist, N)::Matrix
        #scatter(sample[1,:],sample[2,:])
        
        all_samples[(i-1)*N+1:i*N,:] = transpose(sample)
    end
    return all_samples
end

latent_sampler(27720, 2);

In [ ]:
#Latent 1D Sampler
#Makes disconnected 1D uniform samples on unit interval
function latent_sampler(N::Int, modes::Int)
    batch::Int = Int(N/modes)
    all_samples = zeros(Float64, (batch*modes, 1))
    all_labels = zeros(Float64, (batch*modes))
    
    #Fill total samples
    total_samples = batch * modes
    remainder = N - total_samples
    
    cuts::Int = modes * 2 - 1
    
    for i in 1:modes
        a = (2*(i-1)) / cuts
        b = (2*(i-1) + 1) / cuts
        
        dist = Uniform(a, b)
        x = rand(dist, batch)
        labels = ones(batch) * i
        y = zeros(batch)
        
        all_samples[(i-1)*batch+1:i*batch,:] = transpose(x)
        all_labels[(i-1)*batch+1:i*batch] = transpose(labels)
    end
    return all_samples, all_labels
end

x, y = latent_sampler(7200, 5);
x = x'[:,:];

In [ ]:
#Make Latent Distribution
#How to make distribution based on Neural Net of a uniform?
Θ = ones(34)
@dist function generative_dist(a,b,Θ)
    z = uniform(a,b)
end

In [ ]:
#GEN Model
function unpack(Θ)
    W₁ = reshape(Θ[1:8], 8, 1);   
    b₁ = reshape(Θ[9:16], 8)
    
    W₂ = reshape(Θ[17:32], 2, 8); 
    b₂ = reshape(Θ[33:34], 2)
      
    return W₁, b₁, W₂, b₂
end

#Two layer feed-forward network for generation G(z)
function G(z, Θ::AbstractVector)
    if(typeof(z) == Float64)
        z = [z]
    end
    W₁, b₁, W₂, b₂ = unpack(Θ)
    nn = Chain(Dense(W₁, b₁, sigmoid),
               Dense(W₂, b₂))
    return nn(z)
end;

function G(p::Gen.DistWithArgs{Float64}, Θ::Gen.SimpleArg{Any})
    W₁, b₁, W₂, b₂ = unpack(Θ)
    nn = Chain(Dense(W₁, b₁, sigmoid),
               Dense(W₂, b₂))
    return nn(z)
end;

#This is the generative/proposal function
@gen function bnn(n::Int)
    # Create the weight and bias vector.
    k ~ uniform_discrete(2,4)
    Θ = @trace(mvnormal(zeros(34), diagm(ones(34)*sig)), :Θ)
    
    #Draw z from the sampler
    z, _ = latent_sampler(n,k)
    z = z'[:,:]
    #weights = [1/k for i=1:k]
    #cluster_indicator_dist = Categorical(weights)
    
    #cuts::Int = k * 2 - 1
    
    #for j=1:N
        #i = rand(cluster_indicator_dist,1)
        #print(i)
        #a = (2*(i-1)) / cuts
        #b = (2*(i-1) + 1) / cuts
        
        #{(:y, i)} ~ generative_dist(a, b)
    #end
end;

test = bnn(6)
#PyPlot.scatter(test[1,:],test[2,:])

In [153]:
import Random
Θ = ones(34)
include("mixture.jl")
@gen function bnn()
    Θ ~ mvnormal(zeros(34), diagm(ones(34)*sig))
    k ~ uniform_discrete(2,4)
    for i=1:N
        {(:z,i)} ~ neural_dist(k, G, Θ)
    end
end
trace = Gen.simulate(bnn,())
display(trace[:k])
display(trace[(:z,1)])

4

2-element Array{Float64,1}:
 0.3840899044402203
 4.025180048292821

In [ ]:
#Likelihood for Data

function likelihood(x)
    μ₁ = [0.25, 0.25]
    μ₂ = [0.25, 0.75]
    means = [μ₁, μ₂]
    Σ = [[σ, 0] [0, σ]]::Matrix
    dist1 = MvNormal(μ₁, Σ)
    dist2 = MvNormal(μ₂, Σ)
    score = 1
    
    for i in 1:length(x[:,1])
        dist1score = (pdf(dist1,data[i,:]))
        dist2score = (pdf(dist2,data[i,:]))
        update = log((dist1score + dist2score)/2)
        score += update
    end
    return score
end
likelihood(data[:,:])

traces = [Gen.simulate(bnn, (N,)) for i in 1:10]
k = [trace[:k] for trace in traces]
Θ = [trace[:Θ] for trace in traces];
theta_mean = mean(Θ)

In [159]:
#Set Observations for Traces
obs = choicemap()::ChoiceMap
Θ = ones(34)
k = 4
sample, _ = latent_sampler(N,k)
sample = sample'[:,:]
data = G(sample,Θ)
display(typeof(obs))
    for i in 1:N
        obs[(:z, i)] = data[:,i]
    end

function do_inference(num_iters)
    (trace,) = generate(bnn, (), obs)
    acc = []
    ps = []
    for i=1:num_iters
        #(trace, accepted) = hmc(trace, select(:Θ), L=10, eps=0.1)
        (trace, accepted) = mh(trace, select(:Θ,:k), check=false, observations=obs)
        Θ = trace[:Θ]
        k = trace[:k]
        sample, _ = latent_sampler(N,k)
        sample = sample'[:,:]
        data = G(sample,Θ)
        for j in 1:N
            obs[(:z, j)] = data[:,j]
        end
        push!(ps, trace)
        push!(acc, accepted)
        if accepted
            println("Yes")
        end
        if i%10000 == 0
            println("Epoch $i Acceptance Prob: $(sum(acc)/length(acc))")
            #scatter(data[1,:],data[2,:])
            #print(trace[:Θ])
        end
    end
    return ps, acc
end;
do_inference(100000);

DynamicChoiceMap

Epoch 10000 Acceptance Prob: 0.0


InterruptException: InterruptException:

In [157]:
print(acc)

UndefVarError: UndefVarError: acc not defined

In [122]:
trace, = generate(bnn, (), obs)
print(trace[:k])
(new_trace, all) = regenerate(trace,(),(),select(:Θ,:k))
print(new_trace[:k])
#get_args(trace)
#z, _ = latent_sampler(10, 2)
#Θ = ones(34)
#z = z'[:,:]
#z[:,1]
#G(z[:,1],Θ)

32

In [131]:
obs

│
├── (:z, 1647) : [0.1272969953606903, 0.2031534371259947]
│
├── (:z, 3065) : [0.3127161065906352, 0.7619603490916338]
│
├── (:z, 2053) : [0.24859277684485354, 0.6817345482472607]
│
├── (:z, 606) : [0.24664858312901605, 0.25559014615435294]
│
├── (:z, 1035) : [0.24900381935876753, 0.22413536896254901]
│
├── (:z, 847) : [0.25884375772120355, 0.2662570371949716]
│
├── (:z, 1496) : [0.30063183406156485, 0.3205923415827537]
│
├── (:z, 2471) : [0.23157008269853535, 0.8276662207491486]
│
├── (:z, 983) : [0.25298464855881675, 0.2290614709563117]
│
├── (:z, 3343) : [0.25645070716465274, 0.7510959724596861]
│
├── (:z, 3282) : [0.2093630710481057, 0.7879130910508996]
│
├── (:z, 329) : [0.17330464096434783, 0.28928886825118105]
│
├── (:z, 1554) : [0.25152609031762624, 0.35745980078713435]
│
├── (:z, 2897) : [0.19106311960414407, 0.8188778732742898]
│
├── (:z, 2698) : [0.2694019838466002, 0.7587964139014634]
│
├── (:z, 1309) : [0.2536102484004661, 0.27015653165101255]
│
├── (:z, 1273) : [0.154413

In [ ]:
test = get_args(preds[1])
params = preds[1][:Θ]
z, _ = latent_sampler(N, 2)
z = z'[:,:]
predicted_values = G(z, params)
PyPlot.scatter(predicted_values[1,:],predicted_values[2,:])
PyPlot.scatter(data[:,1],data[:,2])

In [ ]:
test = generate(bnn, (N,)::Tuple)
test[1][:Θ]